In [1]:
import numpy as np
import pandas as pd
from confluent_kafka import Producer
from connect import get_connection
from random import randint
from simulation import transaktion_factory
from time import sleep
import socket


from confluent_kafka import Consumer

# Einlesen des Buchbestands

In [2]:
df_bestand = pd.read_csv('data/Bestand.csv', sep=';', encoding='unicode_escape')

In [3]:
bestand_cols=df_bestand.columns

# Einlesen der Tabellen des Producers

In [4]:
c = Consumer({
    'bootstrap.servers': '0.0.0.0:9092',
    'group.id': socket.gethostname(),
    'auto.offset.reset': 'latest'
})


In [26]:
prod_Transaktion_cols={
    0:'Aktion',
    1:'Datum',
    2:'Fernleihe',
    3:'ID_Exemplar',
    4:'ID_Kunde',
    5:'Titel',
    6:'Autor',
    7:'Jahr',
    8:'Art',
    9:'Kennung',
   10:'Zugriffsort'
}

prod_Bewertung_cols={
    0:'Aktion',
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

prod_Neukunden_cols={
    0:'Aktion',
    1:'ID_Kunde',
    #2:'c_count',
    3:'Kundennr',
    4:'Vorname_1',
    5:'Vorname_2',
    6:'Nachname',
    7:'Anrede',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'Titel',
   17:'PersoNr',
   18:'PersoValidTo' 
}

cols=[prod_Transaktion_cols,prod_Bewertung_cols,prod_Neukunden_cols]
topics=["Transaktion", "Bewertung", "Neukunden"]

dfd={}

In [27]:
for col, topic in zip(cols, topics):
    print(col, topic)

    dfd[topic] = pd.DataFrame(columns=list(col.values()))
    
    
    c.subscribe([topic])
    
    for i in range(60):
        msg = c.poll(1)
        if msg is None:
            continue
    
        mrow = pd.DataFrame([msg.value().decode('utf-8').split(";")]).rename(columns=col)

        dfd[topic]=pd.concat([dfd[topic], mrow])

    dfd[topic] = dfd[topic].reset_index(drop=True)
    #c.close()

{0: 'Aktion', 1: 'Datum', 2: 'Fernleihe', 3: 'ID_Exemplar', 4: 'ID_Kunde', 5: 'Titel', 6: 'Autor', 7: 'Jahr', 8: 'Art', 9: 'Kennung', 10: 'Zugriffsort'} Transaktion
{0: 'Aktion', 1: 'ID_Kunde', 2: 'ID_Buch', 3: 'Wertung', 4: 'Rezension'} Bewertung
{0: 'Aktion', 1: 'ID_Kunde', 3: 'Kundennr', 4: 'Vorname_1', 5: 'Vorname_2', 6: 'Nachname', 7: 'Anrede', 8: 'PLZ', 9: 'Strasse', 10: 'Hausnr', 11: 'Mail', 12: 'Tel', 13: 'Geschlecht', 14: 'Geburtsdatum', 15: 'Beruf', 16: 'Titel', 17: 'PersoNr', 18: 'PersoValidTo'} Neukunden


In [28]:
dfd['Transaktion']

,Aktion,Datum,Fernleihe,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Leihe,2025-01-17 02:00:00,False,163,966,Jane Eyre,Charlotte Brontë,1847.0,Roman,XJj49163,xWc37561
1,Leihe,2025-01-17 02:00:00,False,190,5818,Das rote Zimmer,August Strindberg,1879.0,Roman,OSK50471,Qdw65418
2,Leihe,2025-01-17 03:00:00,False,28,5656,Der moderne Kapitalismus,Werner Sombart,1902.0,Sachbuch,XKb87214,zjr62517
3,Rückgabe,2025-01-17 08:00:00,None,28,4656,Der Aufstand der Massen,José Ortega y Gasset,1930.0,Sachbuch,CAN86504,ijy86715
4,Leihe,2025-01-17 11:00:00,False,55,3527,Der Komplize,György Konrád,1983.0,Roman,AMr16975,qkW52709
5,Leihe,2025-01-17 13:00:00,False,69,2613,Hölderlin,Peter Härtling,1976.0,Roman,LdO41780,AGs87436
6,Leihe,2025-01-17 14:00:00,False,58,2517,Aggression,Friedrich Hacker,1971.0,Sachbuch,ByG91465,jya70863
7,Leihe,2025-01-17 15:00:00,False,141,5676,Wie eine Träne im Ozean,Manès Sperber,1961.0,Roman,QaE40513,NCq35976
8,Leihe,2025-01-17 15:00:00,False,275,5666,Der Untergang des Abendlandes,Oswald Spengler,1918.0,Sachbuch,DrB70831,pZn23405
9,Rückgabe,2025-01-17 20:00:00,None,128,1072,Die Kultur der Renaissance in Italien,Jacob Burckhardt,1860.0,Sachbuch,HMu15360,zqb41952


In [29]:
dfd['Bewertung']

,Aktion,ID_Kunde,ID_Buch,Wertung,Rezension
0,Bewertung,4,860,7,"Ein gelungenes Buch, das mich unterhalten hat..."
1,Bewertung,8,151,9,Dieses Buch hat mich beeindruckt! Es ist fast...
2,Bewertung,32,948,5,Das Buch war durchschnittlich. Einige Teile w...
3,Bewertung,85,166,4,Langweilig und vorhersehbar.
4,Bewertung,187,354,5,"Ein durchschnittliches Buch, das weder besond..."
5,Bewertung,15,770,3,Die Geschichte kam nie wirklich in Fahrt. Ich...
6,Bewertung,32,614,9,Ich war beeindruckt von der Tiefe der Charakt...
7,Bewertung,199,223,10,Dieses Buch ist ein wahres Meisterwerk! Es ve...
8,Bewertung,70,787,6,Ich mochte die Grundidee und einige der Chara...
9,Bewertung,111,853,2,Zäh und uninteressant.


In [31]:
dfd['Neukunden']

,Aktion,ID_Kunde,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo,2
0,Neukunde,62,6678,Noah,nan,Kaiser,Herr,37620,Im Winkel,2.0,noah.kaiser@web.de,05533/409541,M,1944-02-28,Senior/in,None,XVP47503,2025-06-05,314
1,Neukunde,63,5996,Julian,nan,Schröder,Herr,15377,Neue Straße,9.0,julian.schröder@gmail.com,033437/244553,m,05/08/1939,Senior/in,None,RKJ42136,2022-01-03,315
2,Neukunde,64,7378,Sofia,nan,Hoffmann,Fr.,99628,Fliederweg,8.0,sofia.hoffmann@t-online.de,036373/802292,w,24.07.1978,Sektions- und Präparationsassistent/in,None,QAN35490,16.08.2025,316
3,Neukunde,65,8073,Mia,nan,Wolf,Fr.,88524,Rosenstraße,6.0,mia.wolf@t-online.de,07374/395656,w,1986-03-04,Immobilienwirt,None,KRQ91853,2025-06-06,317
4,Neukunde,66,9105,Jonas,nan,Herrmann,Herr,99869,Kirchplatz,8.0,jonas.herrmann@gmail.com,03621/202533,M,29.05.1976,Controller/in,Dr.,TAR62891,2023-03-23,318
5,Neukunde,67,5865,Lara,nan,Wolf,Frau,83565,Königsberger Straße 4,nan,lara.wolf@web.de,08092/439859,w,1957-10-17,Rettungssanitäter/in,None,ZPW90287,2021-05-28,319
6,Neukunde,68,8074,Emily,nan,Schulze,Frau,16259,Buchenweg,8.0,emily.schulze@web.de,033451/320243,w,1942-09-08,Senior/in,None,SPX95312,13/08/2021,320
7,Neukunde,69,3872,Leonie,nan,Mayer,Frau,25712,Hauptstraße,6.0,leonie.mayer@gmail.com,04825/570120,w,1987-07-08,Bauzeichner/in,None,AMH06412,Wed May 18 2022,321
8,Neukunde,70,5321,Ben,nan,Köhler,Hr.,67586,Kirchgasse,7.0,ben.köhler@gmx.net,06733/682676,Männlich,1965-11-03,Maler/in und Lackierer/in,None,XEV31827,2024-02-07,322
9,Neukunde,71,2529,Lena,nan,Krause,Frau,86747,Wilhelmstraße,5.0,lena.krause@t-online.de,09087/143513,w,11.05.1980,Sektions- und Präparationsassistent/in,None,FEB46382,2022-10-17,323


# Cleaning der Tabellen

### Transaktion

In [10]:
# IDs zu int konvertieren
dfd['Transaktion']['ID_Exemplar'] = pd.to_numeric(dfd['Transaktion']['ID_Exemplar']).astype(int)
dfd['Transaktion']['ID_Kunde'] = pd.to_numeric(dfd['Transaktion']['ID_Kunde']).astype(int)
# Jahre zu int konvertieren
dfd['Transaktion']['Jahr'] = pd.to_numeric(dfd['Transaktion']['Jahr']).astype(int)

# Fernleihe zu true oder false konvertieren
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("None"), False, dfd['Transaktion']['Fernleihe'] )
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("False"), False, dfd['Transaktion']['Fernleihe'] )
dfd['Transaktion']['Fernleihe'] = np.where(dfd['Transaktion']['Fernleihe'].astype(str).str.contains("True"), True, dfd['Transaktion']['Fernleihe'] )


### Bewertung

In [11]:
# IDs zu int konvertieren
dfd['Bewertung']['ID_Kunde'] = pd.to_numeric(dfd['Bewertung']['ID_Kunde']).astype(int)
dfd['Bewertung']['ID_Buch'] = pd.to_numeric(dfd['Bewertung']['ID_Buch']).astype(int)
dfd['Bewertung']['Wertung'] = pd.to_numeric(dfd['Bewertung']['Wertung']).astype(int)

### Neukunden

In [37]:
# IDs zu int konvertieren
dfd['Neukunden']['ID_Kunde'] = pd.to_numeric(dfd['Neukunden']['ID_Kunde']).astype(int)
dfd['Neukunden']['Kundennr'] = pd.to_numeric(dfd['Neukunden']['Kundennr']).astype(int)

# Anreden konsistent machen 

dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Fr\.'],'Frau', regex=True)
dfd['Neukunden']['Anrede'] = dfd['Neukunden']['Anrede'].replace(['Hr\.'],'Herr', regex=True)
# Titel: "None" to NaN
dfd['Neukunden']['Titel']  = np.where(dfd['Neukunden']['Titel'].str.contains("None"), np.nan, dfd['Neukunden']['Titel'] )
# Vorname_2: "nan" to NaN
dfd['Neukunden']['Vorname_2']  = np.where(dfd['Neukunden']['Vorname_2'].str.contains("nan"), np.nan, dfd['Neukunden']['Vorname_2'] )
# Hausnr: "nan" to NaN
dfd['Neukunden']['Hausnr'] = dfd['Neukunden']['Hausnr'].astype(str)
dfd['Neukunden']['Hausnr']  = np.where(dfd['Neukunden']['Hausnr'].str.contains("nan"), np.nan, dfd['Neukunden']['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
dfd['Neukunden']['PLZ']    = dfd['Neukunden']['PLZ'].str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Hausnr'] = dfd['Neukunden'].loc[dfd['Neukunden']['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
dfd['Neukunden']['Strasse'] = dfd['Neukunden']['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
dfd['Neukunden']['Hausnr'] = pd.to_numeric(dfd['Neukunden']['Hausnr']).astype(int)

# Geschlecht konsistent machen 
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
dfd['Neukunden']['Geschlecht'] = dfd['Neukunden']['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
dfd['Neukunden']['Geburtsdatum'] = pd.to_datetime(dfd['Neukunden']['Geburtsdatum'],format='mixed', dayfirst=True)
dfd['Neukunden']['PersoValidTo'] = pd.to_datetime(dfd['Neukunden']['PersoValidTo'],format='mixed', dayfirst=True)

### Bestand

In [13]:
# Jahre zu int konvertieren
df_bestand['Jahr'] = pd.to_numeric(df_bestand['Jahr']).astype('Int64', errors='ignore')

### Bestandskunden

In [65]:
df_bestandskunden = pd.read_csv('data/Kunde.csv', sep=',', encoding='unicode_escape')

df_bestandskunden['Aktion'] = pd.Series(['Bestandskunde' for x in range(len(df_bestandskunden.index))])


# IDs zu int konvertieren
df_bestandskunden['ID_Kunde'] = pd.to_numeric(df_bestandskunden['ID_Kunde']).astype(int)
df_bestandskunden['Kundennr'] = pd.to_numeric(df_bestandskunden['Kundennr']).astype(int)

# Anreden konsistent machen 

df_bestandskunden['Anrede'] = df_bestandskunden['Anrede'].replace(['Fr\.'],'Frau', regex=True)
df_bestandskunden['Anrede'] = df_bestandskunden['Anrede'].replace(['Hr\.'],'Herr', regex=True)

df_bestandskunden['Hausnr']  = np.where(df_bestandskunden['Hausnr'].astype(str).str.contains("NaN"), np.nan, df_bestandskunden['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
df_bestandskunden['PLZ']    = df_bestandskunden['PLZ'].astype(str).str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Hausnr'] = df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
df_bestandskunden['Strasse'] = df_bestandskunden['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
df_bestandskunden['Hausnr'] = pd.to_numeric(df_bestandskunden['Hausnr']).astype('Int64')

# Geschlecht konsistent machen 
df_bestandskunden['Geschlecht'] = df_bestandskunden['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
df_bestandskunden['Geschlecht'] = df_bestandskunden['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
df_bestandskunden['Geburtsdatum'] = pd.to_datetime(df_bestandskunden['Geburtsdatum'],format='mixed', dayfirst=True)
df_bestandskunden['PersoValidTo'] = pd.to_datetime(df_bestandskunden['PersoValidTo'],format='mixed', dayfirst=True)



/tmp/ipykernel_1109/4017946161.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['8' '6' '4' '8' '3' '7' '6' '4' '3' '8' '3' '6' '8' '3' '8' '7' '3' '8'
 '2' '8' '1' '5' '4' '3' '3' '5' '2' '8' '5' '7' '7' '3' '7' '7' '9' '1'
 '5' '10' '4' '5' '4' '1' '9' '10' '10' '7' '8' '2' '10' '5' '8' '6' '8'
 '4' '3' '4' '9' '9' '10' '5' '4' '2' '7' '7' '3' '7' '5' '2' '2' '8' '9'
 '8' '3' '4' '3' '10' '10' '3' '10' '5' '9' '5' '7' '10']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Hausnr'] = df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)


In [64]:
df_bestandskunden

,ID_Kunde,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo,Aktion
0,1,8366,Elias,NaN,Kaiser,Herr,24619,MozartstraÃe,8,elias.kaiser@web.de,04323/829874,Männlich,1988-02-22,Product Owner,Dr.,NPE20376,2021-05-27,Bestandskunde
1,2,9208,Elias,NaN,KÃ¶nig,Herr,54574,MÃ¼hlenstraÃe,1,elias.kÃ¶nig@gmail.com,06594/313132,Männlich,1997-07-28,Sektions- und PrÃ¤parationsassistent/in,NaN,GMN18965,2024-01-26,Bestandskunde
2,3,3837,Moritz,NaN,MÃ¼ller,Herr,01097,EichendorffstraÃe,8,moritz.mÃ¼ller@gmail.com,0351/156882,Männlich,1963-08-20,Data Analyst,NaN,DNS52984,2022-11-05,Bestandskunde
3,4,7306,Leonie,NaN,Herrmann,Frau,23923,Marktplatz,1,leonie.herrmann@web.de,038821/70723,Weiblich,1952-07-07,Senior/in,NaN,IXU64239,2025-03-27,Bestandskunde
4,5,4589,Elias,NaN,KrÃ¼ger,Herr,55469,Kirchweg,4,elias.krÃ¼ger@gmail.com,06766/872686,Männlich,1995-07-13,Landwirt/in,NaN,MJB45392,2025-01-15,Bestandskunde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,5019,Louis,NaN,Schulze,Herr,25712,RÃ¶merstraÃe,7,louis.schulze@gmail.com,04825/579613,Männlich,2006-01-27,Bauzeichner/in,NaN,XGP73984,2025-03-31,Bestandskunde
496,497,6603,Sophie,NaN,Braun,Frau,16259,Neue,10,sophie.braun@t-online.de,033451/330710,Weiblich,1967-04-02,Finanzwirt/in,NaN,MVR46513,2023-10-21,Bestandskunde
497,498,4465,Philipp,NaN,Kaiser,Herr,08606,Breslauer,9,philipp.kaiser@t-online.de,037463/768172,MÃ¤nnlich,1940-02-16,Senior/in,NaN,FIQ27658,2020-08-02,Bestandskunde
498,499,6695,Elias,NaN,Hoffmann,Herr,19230,Lange,10,elias.hoffmann@gmx.net,038856/699772,Männlich,1950-01-15,Senior/in,Prof.,CNG07613,2020-05-22,Bestandskunde


In [59]:
dfd['Neukunden']

,Aktion,ID_Kunde,Kundennr,Vorname_1,Vorname_2,Nachname,Anrede,PLZ,Strasse,Hausnr,Mail,Tel,Geschlecht,Geburtsdatum,Beruf,Titel,PersoNr,PersoValidTo,2
0,Neukunde,62,6678,Noah,NaN,Kaiser,Herr,37620,Im,2,noah.kaiser@web.de,05533/409541,Männlich,1944-02-28,Senior/in,NaN,XVP47503,2025-06-05,314
1,Neukunde,63,5996,Julian,NaN,Schröder,Herr,15377,Neue,9,julian.schröder@gmail.com,033437/244553,Männlich,1939-08-05,Senior/in,NaN,RKJ42136,2022-01-03,315
2,Neukunde,64,7378,Sofia,NaN,Hoffmann,Frau,99628,Fliederweg,8,sofia.hoffmann@t-online.de,036373/802292,Weiblich,1978-07-24,Sektions- und Präparationsassistent/in,NaN,QAN35490,2025-08-16,316
3,Neukunde,65,8073,Mia,NaN,Wolf,Frau,88524,Rosenstraße,6,mia.wolf@t-online.de,07374/395656,Weiblich,1986-03-04,Immobilienwirt,NaN,KRQ91853,2025-06-06,317
4,Neukunde,66,9105,Jonas,NaN,Herrmann,Herr,99869,Kirchplatz,8,jonas.herrmann@gmail.com,03621/202533,Männlich,1976-05-29,Controller/in,Dr.,TAR62891,2023-03-23,318
5,Neukunde,67,5865,Lara,NaN,Wolf,Frau,83565,Königsberger,4,lara.wolf@web.de,08092/439859,Weiblich,1957-10-17,Rettungssanitäter/in,NaN,ZPW90287,2021-05-28,319
6,Neukunde,68,8074,Emily,NaN,Schulze,Frau,16259,Buchenweg,8,emily.schulze@web.de,033451/320243,Weiblich,1942-09-08,Senior/in,NaN,SPX95312,2021-08-13,320
7,Neukunde,69,3872,Leonie,NaN,Mayer,Frau,25712,Hauptstraße,6,leonie.mayer@gmail.com,04825/570120,Weiblich,1987-07-08,Bauzeichner/in,NaN,AMH06412,2022-05-18,321
8,Neukunde,70,5321,Ben,NaN,Köhler,Herr,67586,Kirchgasse,7,ben.köhler@gmx.net,06733/682676,Männlich,1965-11-03,Maler/in und Lackierer/in,NaN,XEV31827,2024-02-07,322
9,Neukunde,71,2529,Lena,NaN,Krause,Frau,86747,Wilhelmstraße,5,lena.krause@t-online.de,09087/143513,Weiblich,1980-05-11,Sektions- und Präparationsassistent/in,NaN,FEB46382,2022-10-17,323


In [14]:
dfd['Transaktion']

,Aktion,Datum,Fernleihe,ID_Exemplar,ID_Kunde,Titel,Autor,Jahr,Art,Kennung,Zugriffsort
0,Leihe,2025-01-14 19:00:00,False,295,5508,Love Story,Erich Segal,1970,Roman,bwc42965,wVE86507
1,Leihe,2025-01-14 19:00:00,False,179,2908,"Gödel, Escher, Bach – ein Endloses Geflochten...",Douglas R. Hofstadter,1979,Sachbuch,rKS31970,rWu13725
2,Rückgabe,2025-01-15 00:00:00,False,7,4187,Die Asche meiner Mutter,Frank McCourt,1996,Roman,CwE93710,plV36087
3,Leihe,2025-01-15 02:00:00,False,282,4946,Der Pate,Mario Puzo,1969,Roman,CdH54832,yFl09845
4,Leihe,2025-01-15 05:00:00,False,168,2712,Deutschland. Ein Wintermärchen,Heinrich Heine,1844,Epos,lkS13928,MOr03729
5,Leihe,2025-01-15 07:00:00,False,36,3314,Schindlers Liste,Thomas Keneally,1982,Roman,BdD79820,Sbn62495
6,Rückgabe,2025-01-15 12:00:00,False,37,1918,Kleiner Mann – was nun?,Hans Fallada,1932,Roman,Osy15037,lkW13504
7,Leihe,2025-01-15 12:00:00,False,198,2899,Struwwelpeter,Heinrich Hoffmann,1845,Kinderbuch,nRG51072,Dux16349
8,Leihe,2025-01-15 12:00:00,False,120,2445,Der arme Spielmann,Franz Grillparzer,1847,Novelle,uyI14523,HTR95106
9,Leihe,2025-01-15 15:00:00,False,168,6562,Eifel-Liebe,Jacques Berndorf,2002,Roman,qBZ67804,CFU80741


# Aufbau der Tabellen für das Core DWH

In [15]:
dfs={}

In [16]:
Kunde_cols={
    0:'ID_Kunde',
    1:'ID_Mitgliedsstatus',
    2:'Kundennr',
    3:'Vorname_1',
    4:'Vorname_2',
    5:'Nachname',
    6:'Anrede',
    7:'Titel',
    8:'PLZ',
    9:'Strasse',
   10:'Hausnr',
   11:'Mail',
   12:'Tel',
   13:'Geschlecht',
   14:'Geburtsdatum',
   15:'Beruf',
   16:'PersoNr',
   17:'PersoValidTo', 
   18:'Mitglied seit',
   19:'Mitglied bis' 
}

Bewertung_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Buch_cols={
    1:'ID_Kunde',
    2:'ID_Buch',
    3:'Wertung',
    4:'Rezension'
}

Exemplare_cols={
    1:'ID_Bestand',
    2:'Nr',
    3:'Kennung',
    4:'Zugriffsort',
    5:'Zustand'
}

Buch_cols={
    1:'ID_Autor',
    2:'Nr',
    3:'Titel',
    4:'Jahr',
    5:'Art',
}

Leihe_cols={
    0:'ID_Kunde',
    1:'ID_Exemplar',
    2:'Ausleihdatum',
    3:'Rueckgabedatum',
    4:'Verlaengerungsstatus',
    5:'Mahnstatus',
    6:'Fernleihe'
}

Mitgliedsstatus_cols={
    0:'ID_Mitgliedsstatus',
    2:'Bezeichnung',
    3:'Jahresbeitrag',
    4:'Mahnbetrag'
}

Beitragszahlung_cols={
    0:'ID_Zahlung',
    2:'ID_Kunde',
    3:'Datum',
    4:'Betrag',
}

In [17]:
dfs['Kunde']     = dfd['Neukunden'].reindex(columns=list(Kunde_cols.values()))
dfs['Bewertung'] = dfd['Bewertung'].reindex(columns=list(Bewertung_cols.values()))
dfs['Exemplare'] = df_bestand.reindex(columns=list(Exemplare_cols.values())).rename(columns={
                                                                        'ID_Bestand':'ID_Exemplar',
                                                                        'Nr':'ID_Buch'    
                                                                        })

dfs['Autor']   = df_bestand[['Autor', 'Herkunft']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ID_Autor'})

dfs['Buch']    = pd.merge(df_bestand, 
                          dfs['Autor'], 
                          on=['Autor']
                         )[list(Buch_cols.values())]                                  \
                                                    .drop_duplicates()                \
                                                    .reset_index(drop=True)           \
                                                    .rename(columns={'Nr':'ID_Buch'})

dfs['Leihe'] = dfd['Transaktion'].reindex(columns=list(Leihe_cols.values())).reset_index().rename(columns={'index':'ID_Leihe'})


### more ###
dfs['Mitgliedsstatus'] = pd.DataFrame(columns=list(Mitgliedsstatus_cols.values()))
dfs['Beitragszahlung'] = dfd['Neukunden'].reindex(columns=list(Beitragszahlung_cols.values()))


In [18]:
dfs['Leihe'][['Ausleihdatum']]

dfd['Transaktion']

df = dfs['Leihe'].copy()
#df.loc[df["Aktion"] == "Leihe", "gender"] = 1

In [19]:

dfs['Leihe']['Ausleihdatum']=pd.to_datetime(dfs['Leihe']['Ausleihdatum'])
dfs['Leihe'].loc[dfd['Transaktion']['Aktion'] == "Leihe", "Ausleihdatum"] = dfd['Transaktion']['Datum']

dfs['Leihe']['Rueckgabedatum']=pd.to_datetime(dfs['Leihe']['Rueckgabedatum'])
dfs['Leihe'].loc[dfd['Transaktion']['Aktion'] == "Rückgabe", "Rueckgabedatum"] = dfd['Transaktion']['Datum']


In [20]:
dfs['Leihe']

,ID_Leihe,ID_Kunde,ID_Exemplar,Ausleihdatum,Rueckgabedatum,Verlaengerungsstatus,Mahnstatus,Fernleihe
0,0,5508,295,2025-01-14 19:00:00,NaT,NaN,NaN,False
1,1,2908,179,2025-01-14 19:00:00,NaT,NaN,NaN,False
2,2,4187,7,NaT,2025-01-15 00:00:00,NaN,NaN,False
3,3,4946,282,2025-01-15 02:00:00,NaT,NaN,NaN,False
4,4,2712,168,2025-01-15 05:00:00,NaT,NaN,NaN,False
5,5,3314,36,2025-01-15 07:00:00,NaT,NaN,NaN,False
6,6,1918,37,NaT,2025-01-15 12:00:00,NaN,NaN,False
7,7,2899,198,2025-01-15 12:00:00,NaT,NaN,NaN,False
8,8,2445,120,2025-01-15 12:00:00,NaT,NaN,NaN,False
9,9,6562,168,2025-01-15 15:00:00,NaT,NaN,NaN,False


%6|1727373654.407|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Disconnected (after 175203ms in state UP, 1 identical error(s) suppressed)
%6|1727373654.408|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727373654.635|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727373654.652|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configurat

In [ ]:
df.columns

%6|1727427857.981|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727427857.981|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY)
%6|1727427858.255|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Disconnected while requesting ApiVersion: might be caused by incorrect security.protocol configuration (connecting to a SSL listener?) or broker version is < 0.10 (see api.version.request) (after 0ms in state APIVERSION_QUERY, 1 identical error(s) suppressed)
%6|1727427858.27

# Schicke die Tabellen an den SQL Server


In [37]:
from sqlalchemy import create_engine, inspect, text

USER = 'root'
PASSWORD = 'root'
HOST = 'localhost'          # see yml file
PORT = 3307
DATABASE = 'test_db_1'
 

def get_connection(database=DATABASE):
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}"\
            .format(USER, PASSWORD, HOST, PORT, database)
    )

engine = get_connection('mysql')
with engine.connect() as connection:
    insp = inspect(engine)
    db_list = insp.get_schema_names()
    print(f"Connection to the {HOST} for user {USER} created successfully.")
    if DATABASE not in db_list:
        sql = text(f"CREATE DATABASE {DATABASE} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
        result = connection.execute(sql)
        if result:
            print(f"Database {DATABASE} created!")

Connection to the localhost for user root created successfully.


In [38]:
for Tabelle in dfs.keys():
    dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 

In [119]:


CDW_Tabellen_Liste = insp.get_table_names()

Statische_Tabellen = ['Exemplare', 'Autor', 'Buch']

for Tabelle in Statische_Tabellen:
    if Tabelle not in CDW_Tabellen_Liste:
        dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='replace') 

for Tabelle in dfs.keys():
    if Tabelle not in Statische_Tabellen:
        dfs[Tabelle].to_sql(Tabelle, con=engine,schema=DATABASE, index=False, if_exists='append') 

Kunde
Bewertung
Leihe
Mitgliedsstatus
Beitragszahlung


%3|1727373845.031|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 19 identical error(s) suppressed)
%3|1727373868.033|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection refused (after 0ms in state CONNECT, 21 identical error(s) suppressed)
%3|1727373871.073|FAIL|rdkafka#consumer-1| [thrd:GroupCoordinator]: GroupCoordinator: 127.0.0.1:9092: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 3 identical error(s) suppressed)
%3|1727373875.033|FAIL|rdkafka#consumer-1| [thrd:127.0.0.1:9092/1]: 127.0.0.1:9092/1: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT, 16 identical error(s) suppressed)
%3|1727373903.034|FAIL|rdkafka#consumer-1| [thrd:0.0.0.0:9092/bootstrap]: 0.0.0.0:9092/bootstrap: Connect to ipv4#0.0.0.0:9092 failed: Connection r

In [ ]:
df_bestandskunden = pd.read_csv('data/Kunde.csv', sep=',', encoding='unicode_escape')

df_bestandskunden['Aktion'] = pd.Series(['Bestandskunde' for x in range(len(df_bestandskunden.index))])


# IDs zu int konvertieren
df_bestandskunden['ID_Kunde'] = pd.to_numeric(df_bestandskunden['ID_Kunde']).astype(int)
df_bestandskunden['Kundennr'] = pd.to_numeric(df_bestandskunden['Kundennr']).astype(int)

# Anreden konsistent machen 

df_bestandskunden['Anrede'] = df_bestandskunden['Anrede'].replace(['Fr\.'],'Frau', regex=True)
df_bestandskunden['Anrede'] = df_bestandskunden['Anrede'].replace(['Hr\.'],'Herr', regex=True)

df_bestandskunden['Hausnr']  = np.where(df_bestandskunden['Hausnr'].astype(str).str.contains("NaN"), np.nan, df_bestandskunden['Hausnr'] )
# 4-stellige PLZ mit "0" präfixen 
df_bestandskunden['PLZ']    = df_bestandskunden['PLZ'].astype(str).str.strip().str.rjust(5, '0')

# Hausnummern aus Strasse-Spalte extrahieren
def extract_hausnr(strasse):
  if pd.isna(strasse):
    return np.nan
  parts = strasse.split()
  if len(parts) > 1 and parts[-1].isdigit():
    return parts[-1]
  else:
    return np.nan
#Apply extract_hausnr, wenn 'Hausnr' NaN und entferne die Hausnummern aus der Strasse Spalte
df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Hausnr'] = df_bestandskunden.loc[df_bestandskunden['Hausnr'].isnull(), 'Strasse'].apply(extract_hausnr)
df_bestandskunden['Strasse'] = df_bestandskunden['Strasse'].str.rsplit(expand=True)[0]
# Convert Hausnr to int
df_bestandskunden['Hausnr'] = pd.to_numeric(df_bestandskunden['Hausnr']).astype('Int64')

# Geschlecht konsistent machen 
df_bestandskunden['Geschlecht'] = df_bestandskunden['Geschlecht'].str.strip().replace(["^w$", "^W$"],'Weiblich', regex=True)
df_bestandskunden['Geschlecht'] = df_bestandskunden['Geschlecht'].str.strip().replace(["^m$", "^M$"],'Männlich', regex=True)

# Datumsspalten konsistent machen
df_bestandskunden['Geburtsdatum'] = pd.to_datetime(df_bestandskunden['Geburtsdatum'],format='mixed', dayfirst=True)
df_bestandskunden['PersoValidTo'] = pd.to_datetime(df_bestandskunden['PersoValidTo'],format='mixed', dayfirst=True)


df_bestandskunden = df_bestandskunden.reindex(columns=list(Kunde_cols.values()))

